#### context switching
when switching between processess, there are a few bits of information that need to be saved:
* %rip (the register instruction pointer)
    * this is the pointer that is pointing to the next instruction that the cpu will execute

THe code section of memory is read only.

Process:  
|Process|
|-|
|Stack|
|stack contents|
|heap|
|data|
|code|
|grows in this direction (downward)|

### Registers:  
There are 16 general purpose registers in 64-bit x86 architecture.   
There are 8 general purpose registers in 32-bit x86 architecture.

%rbp(%ebp) is used as a **frame/base pointer** (Bottom of the stack).  
%rsp(%esp) is used as a **stack pointer** (Top of the stack).

### Stack:  
The main feature of operations on stack: LIFO (Last In First Out) / FILO (First In Last Out)  
Instructions that modifies stack include: **Push** and **Pop** 
* Push moves the stack pointer down 4 bits in 32bit (8 in 64bit) and sets the value of the new address the stack pointer is pointing to whatever was passed in to 
* Pop sets the value of whatever was passed in to the value of the stack pointer, then moves the stack pointer up 4 in 32bit (8 in 64bit)

### Function Calls:  
Pass control. Instructions pass control between caller and collee %rip and CPU.  
* RIP remains on the next executable line in main().
* Old Base pointer stored. Call and returns instruction, both alter stack and instruction pointer.  

Pass data: What's the data exchange between caller and callee?  
*  The return value is always put into %rax

How does x86 implement "Pass data" for function calls?


Why do the programs sometimes need to save register values before using the register?

What are caller-saved registers?

What are calle-saved registers?

When the CPU needs to switch the current process to another one, what information related to the process needs to be preserved? So next time when it becomes active it can continue from where it was "paused".